In [ ]:
import json
import pandas as pd

def filter_by_withn(tactics_df: pd.DataFrame):
    filtered_tactics_df = pd.DataFrame(columns=tactics_df.columns.values.tolist())
    for index, row in tactics_df.iterrows():
        value = row['state_before']
        if "with\n" in value:
            filtered_tactics_df.loc[len(filtered_tactics_df)] = row
            tactics_df.drop(index=index, inplace=True)
            print(row["positive"])
            
        
    return tactics_df, filtered_tactics_df
    
with open("/home/vincentzhu/gfn_ntp/data/post_verify_counting_negatives/negative_tactics_dataset.json") as f:
    data = json.load(f)
    
print(len(data))
# print(data)
data_df = pd.DataFrame.from_dict(data)
# print(type(data))
# print(data.columns.values.tolist())
filtered_df, filtered_false_neg_df = filter_by_withn(data_df)
# print(filtered_df)
list_of_dicts = filtered_df.to_dict(orient='records')
list_of_dicts2 = filtered_false_neg_df.to_dict(orient='records')

# with open("/home/vincentzhu/gfn_ntp/data/post_verify_counting_negatives/filtered_neg_tactics_data.json", "w") as f:
#     data = json.dump(list_of_dicts, f, indent=4)
# with open("/home/vincentzhu/gfn_ntp/data/post_verify_counting_negatives/filtered_false_neg_data.json", "w") as f:
#     data = json.dump(list_of_dicts2, f, indent=4)

with open("/home/vincentzhu/gfn_ntp/data/post_verify_counting_negatives/filtered_neg_tactics_data.json") as f:
    data1 = json.load(f)
with open("/home/vincentzhu/gfn_ntp/data/post_verify_counting_negatives/filtered_false_neg_data.json") as f:
    data2 = json.load(f)
    
print(len(data1))
print(len(data2))

In [16]:
import json
import pandas as pd
from transformers import AutoTokenizer

# tokenizer_id = "EleutherAI/llemma_7b"
tokenizer_id = "deepseek-ai/DeepSeek-Prover-V1.5-RL"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)

with open("/home/vincentzhu/gfn_ntp/data/time_filtered_v2.json") as f:
    data = json.load(f)

print("Length of original data: ", len(data))
og_len = len(data)

data_k_v = data.items()
num_entries_to_remove = int(len(data) * 0.05)

def count_tokens(text):
    return len(tokenizer.tokenize(text))

sorted_data = sorted(
    data_k_v, 
    key=lambda x: count_tokens(x[1]["traced_tactics"][0]["state_before"]), 
    reverse=True
)

keys_to_remove = [key for key, _ in sorted_data[:num_entries_to_remove]]
for key in keys_to_remove:
    del data[key]


print("Length of filtered data: ", len(data))
print("Percentage Kept: ", (len(data) / og_len))

with open("/home/vincentzhu/gfn_ntp/data/time_filtered_v3.json", "w") as f:
    data = json.dump(data, f, indent=4)

Length of original data:  12092
Length of filtered data:  11488
Percentage Kept:  0.9500496195831954


In [3]:
import json
import pickle

with open("/home/vincentzhu/gfn_ntp/data/time_filtered_v3.json") as f:
    data = json.load(f)
    
pickle_path = "/home/vincentzhu/gfn_ntp/data/v3_wfs_pickle.pickle"
pickle.dump(data, open(pickle_path, "wb"))
pickle.load(data, open(pickle_path, "wb"))
    
print(len(data))

for k, v in data.items():
    print(k)
    for key, value in v.items():
        print(key, value)
    break

11488
0
url https://github.com/leanprover-community/mathlib4
commit 29dcec074de168ac2bf835a77ef68bbe069194c5
file_path Mathlib/Data/Matrix/Block.lean
full_name Matrix.toBlocks₂₂_diagonal
start [310, 1]
end [315, 61]
traced_tactics [{'tactic': 'unfold toBlocks₂₂', 'annotated_tactic': ['unfold <a>toBlocks₂₂</a>', [{'full_name': 'Matrix.toBlocks₂₂', 'def_path': 'Mathlib/Data/Matrix/Block.lean', 'def_pos': [93, 5], 'def_end_pos': [93, 15]}]], 'state_before': "l : Type u_1\nm : Type u_2\nn : Type u_3\no : Type u_4\np : Type u_5\nq : Type u_6\nm' : o → Type u_7\nn' : o → Type u_8\np' : o → Type u_9\nR : Type u_10\nS : Type u_11\nα : Type u_12\nβ : Type u_13\ninst✝² : DecidableEq l\ninst✝¹ : DecidableEq m\ninst✝ : Zero α\nv : l ⊕ m → α\n⊢ (diagonal v).toBlocks₂₂ = diagonal fun i => v (Sum.inr i)", 'state_after': "l : Type u_1\nm : Type u_2\nn : Type u_3\no : Type u_4\np : Type u_5\nq : Type u_6\nm' : o → Type u_7\nn' : o → Type u_8\np' : o → Type u_9\nR : Type u_10\nS : Type u_11\nα : Type u_

In [18]:
import json
import random
import pandas as pd
from transformers import AutoTokenizer

# tokenizer_id = "EleutherAI/llemma_7b"
tokenizer_id = "deepseek-ai/DeepSeek-Prover-V1.5-RL"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)

def sample_small_test_set(data: dict, num_samples: int, seed: int = None):
    if seed is not None:
        random.seed(seed)
        
    sampled_keys = random.sample(list(data.keys()), num_samples)
    sampled_data = {key: data[key] for key in sampled_keys}
    return sampled_data

def count_tokens(text):
    return len(tokenizer.tokenize(text))

with open("/home/vincentzhu/gfn_ntp/data/time_filtered_v3_wfs.json") as f:
    data = json.load(f)
    
# v = data.values()
# max_tokens = 0
# for entry in v:
#     max_tokens = max(count_tokens(entry["traced_tactics"][0]["state_before"]), max_tokens)
# print(max_tokens)

sampled_data = sample_small_test_set(data, 30, 42)

with open("/home/vincentzhu/gfn_ntp/data/time_filtered_v3_wfs_30_samples.json", "w") as f:
    data = json.dump(sampled_data, f, indent=4)

print(len(sampled_data))

print("Formatted with json.dumps:")
print(json.dumps(sampled_data, indent=4))

sampled_data_df = pd.DataFrame.from_dict(sampled_data, orient='index')

30
Formatted with json.dumps:
{
    "26176": {
        "url": "https://github.com/leanprover-community/mathlib4",
        "commit": "29dcec074de168ac2bf835a77ef68bbe069194c5",
        "file_path": "Mathlib/Data/Finsupp/ToDFinsupp.lean",
        "full_name": "sigmaFinsuppEquivDFinsupp_smul",
        "start": [
            358,
            1
        ],
        "end": [
            362,
            6
        ],
        "traced_tactics": [
            {
                "tactic": "ext",
                "annotated_tactic": [
                    "ext",
                    []
                ],
                "state_before": "\u03b9 : Type u_1\nR\u271d : Type u_2\nM : Type u_3\n\u03b7 : \u03b9 \u2192 Type u_4\nN : Type u_5\ninst\u271d\u00b3 : Semiring R\u271d\nR : Type u_6\ninst\u271d\u00b2 : Monoid R\ninst\u271d\u00b9 : AddMonoid N\ninst\u271d : DistribMulAction R N\nr : R\nf : (i : \u03b9) \u00d7 \u03b7 i \u2192\u2080 N\n\u22a2 sigmaFinsuppEquivDFinsupp (r \u2022 f) = r \u2022 sigmaFinsuppE